In [ ]:
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 3.2 MB/s eta 0:00:00


In [ ]:
import re
import spacy
import docx
import os
import pandas as pd

In [ ]:
def paragraph_contains_table(paragraph):
    for run in paragraph.runs:
        for element in run._r:
            if element.tag.endswith("tbl"):
                return True
    return False


def extract_text_from_docx(file_path):
    doc = docx.Document(file_path)

    full_text = []

    for paragraph in doc.paragraphs:
        full_text.append(paragraph.text)

    return '\n'.join(full_text)


def preprocess_text(plain_text):

  # Sample text (your extracted text)
  text = plain_text

  # Remove dates using regular expressions
  text = re.sub(r'\d{1,2}[\s/]\d{1,2}[\s/]\d{2,4}', '', text)

  # Remove addresses using regular expressions
  address_pattern = r'\b\d+\s+\w+\s+\w+,(\s+\w+)+\s+\d{5}\b'
  text = re.sub(address_pattern, '', text)

  # Lowercase the text
  # text = text.lower()

  # Tokenize into words (you can use more advanced tokenization if needed)
  words = text.split()

  # Remove common stop words (you can use a predefined list of stop words)
  stop_words = set(['and', 'the', 'in', 'of', 'for', 'to', 'a','"'])
  filtered_words = [word for word in words if word not in stop_words]

  # Reconstruct the text
  filtered_text = ' '.join(filtered_words)
  filtered_text = re.sub(r'[^a-zA-Z\s]', '', filtered_text)
  filtered_text = ' '.join(filtered_words)

  return filtered_text




def extract_desired_doc(filtered_text):

  sentences = filtered_text

  # Load a spaCy NLP model (English)
  nlp = spacy.load("en_core_web_sm")

  # Define the possible stop words
  stop_words = ["lessor", "licensor", "owner", "landlord"]

  # Process and extract words from sentences

      # Tokenize the sentence
  doc = nlp(filtered_text)

  # Initialize variables to track the extracted words
  extracting_words = False
  extracted_words = []

  # Iterate through the tokens and apply the extraction logic
  for token in doc:

    if "between" in token.text.lower():
          extracting_words = True
    elif extracting_words and token.text.lower() in stop_words:
        extracting_words = False
        break
    elif extracting_words:
        extracted_words.append(token.text)

      # Join the extracted words and print
  extracted_text = " ".join(extracted_words)
  extracted_text = extracted_text.replace(" / ", "/")
  # print("Extracted Text:", extracted_text)

  return extracted_text

def final_extract_name(extracted_text):

  stop_words = ["aged", "residing","age", "with","s/o", "d.no","plat","pan", "no"]
  stop_symbols = ["#",",","(",";"]

  # Process and extract words from sentences

      # Tokenize the sentence into words
  words = extracted_text.split()

  # Initialize a list to store the extracted words
  extracted_words = []

  # Iterate through the words and apply the extraction logic
  for word in words:
      # Check if the current word is a stop word or a stop symbol
      if word.lower() in stop_words or word in stop_symbols:
          break  # Stop when a stop word or symbol is encountered
      extracted_words.append(word)

  # Join the extracted words and print
  extracted_text = " ".join(extracted_words)

  return extracted_text

In [ ]:
folder_path = "Testing_words"
docx_path = os.listdir(folder_path)
ans_dict = {}
i = 0
for file_path in docx_path:
  try:
    print(file_path)
    doc_path = folder_path + "/" + file_path
    plain_text = extract_text_from_docx(doc_path)
    filtered_text = preprocess_text(plain_text)
    extracted_text = extract_desired_doc(filtered_text)
    # print(extracted_text)
    extract_name = final_extract_name(extracted_text)
    extract_name = extract_name.replace(": ","")
    extract_name = extract_name.replace("wef ","")
    extract_name = extract_name.replace("Mr. ", '')
    extract_name = extract_name.replace("Mr .", '')
    extract_name = extract_name.replace("Mrs. ", '')
    extract_name = extract_name.replace("Smt ", '')
    extract_name = extract_name.replace("Sri ", '')
    ans_dict[file_path] = extract_name
    print(extract_name)
  except:
    ans_dict[file_path] = extract_name
    print(file_path)


EVAL_Rental_Agreement_5.docx
C.BHAGYAMMA
EVAL_Rental_Agreement_2.docx
S Parthasarathy
EVAL_Rental_Agreement_6.docx
KAPIL MEHROTRA
EVAL_Rental_Agreement_3.docx
S.Sakunthala
EVAL_Rental_Agreement_7.docx

EVAL_Rental_Agreement_4.docx
V.K.NATARAJ
EVAL_Rental_Agreement_8.docx
Giddappa
EVAL_Rental_Agreement_1.docx
Hanumaiah


In [ ]:
df_ans = pd.DataFrame(ans_dict.items(), columns=['ID', 'PartyOne'])
# df_ans['ID'] = df_ans['ID'].str.replace('EVAL_Rental_','Testing_')
# df_ans['ID'] = df_ans['ID'].str.replace('Agreement_','Agrrement_File')
df_ans['ID'] = df_ans['ID'].str.replace('.docx','')

<ipython-input-105-08ea5d399e6d>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_ans['ID'] = df_ans['ID'].str.replace('.docx','')


In [ ]:
df_ans

,ID,PartyOne
0,EVAL_Rental_Agreement_5,C.BHAGYAMMA
1,EVAL_Rental_Agreement_2,S Parthasarathy
2,EVAL_Rental_Agreement_6,KAPIL MEHROTRA
3,EVAL_Rental_Agreement_3,S.Sakunthala
4,EVAL_Rental_Agreement_7,
5,EVAL_Rental_Agreement_4,V.K.NATARAJ
6,EVAL_Rental_Agreement_8,Giddappa
7,EVAL_Rental_Agreement_1,Hanumaiah


In [ ]:
df_ans.to_csv('rent_submission.csv',index=False)

In [ ]:
# file_path = "/content/Training_words/9800011_Rental_Agreement_11.docx"
# doc_path = folder_path + "/" + file_path
doc_path = "Testing_words/EVAL_Rental_Agreement_8.docx"
plain_text = extract_text_from_docx(doc_path)
print(f"Plain:{plain_text}")
filtered_text = preprocess_text(plain_text)
print(f"filtered_text:{filtered_text}")
extracted_text = extract_desired_doc(filtered_text)
print(f"extracted_text:{extracted_text}")
extract_name = final_extract_name(extracted_text)
print(f"extract_name:{extract_name}")


Plain:STAMP DUTY KARNATAKA
c$j5Coz3f$ 3oc>rtjs s&o^c^ost ^C7e)©c> &sd: &&. 2/
ds	afss&de dTOjSczSrt eroaiaieef«^*^x
This sheet can be used for any document _ . , ,
rossssSrfrS d,
WR E N T A L AGREEMENT”
This Rental Agreement is made and Executed on
at Bangalore, by and Between:- wef 1/4/2014
Mr .Giddappa S/o lute Nagappa
D No.134 2nd cross
Shunthi L/o Rumamurthynagar
Bangalore 560016
Hereinafter Called the "OWNER" of the One Part: Ms Pottumurthi Udayalaxmi W/o Ramakr ishna Kajjam
H No.27-7 Main Bazar,
Miryalaguda, i, Nalgonda Dist A p 503207.
Hereinafter called the "TENANT" of the Other Part witnesseth as follows terms and conditions.
1. Whereas the Tenant has paid Advance amount of Rs.40,000/- ( Forty thousand only	).; to the owner.
This amount shall not carry any interest and this is refundable at the time vacating.. If any arrears balance that charges will deducted from the advance amount at the time of vacating.
has agreed to pay the Rental Charges of thousand three hundred only	) 

In [ ]:
doc_path = "Testing_words/EVAL_Rental_Agreement_7.docx"
plain_text = extract_text_from_docx(doc_path)
print(f"Plain:{plain_text}")
filtered_text = preprocess_text(plain_text)
print(f"filtered_text:{filtered_text}")
extracted_text = extract_desired_doc(filtered_text)
print(f"extracted_text:{extracted_text}")
extract_name = final_extract_name(extracted_text)
print(f"extract_name:{extract_name}")

Plain:rental aaremeant


Whereas the terms OWNER and TENANT shall mean and include their respective heirs, legal representatives, administrators, eexecutors and assigns.
Whereas the OWNER is sole and absolute owner of the schedule property described hereunder and where as the owner is willing to let-out this said house to the TENANT on rental basis for a monthly rent of Rs. 9000/- (Rupees Nine Thousand Only) and whereas TENANT is willing to take the said house and agreeing to pay the rent to the owner on or before 5th of every english calender month and the rent is revisable after 11 months.
The rent executed is for a period of 11 (Eleven) months effective from 07-072014 whereas the TENANT has paid an interest free advance of Rs. 90,000/- (Rupees Ninety Thousand Only) which has been receipted by the OWNER seperately. The advance amount is refundable to the TENANT on vacation of the house in good and tenantable condition.
Whereas the TENANT shall not sub-let/under-let the house to any p